In [4]:
import pandas as pd
from sodapy import Socrata
import folium

import secrets

In [5]:
## Initialize the client
client = Socrata("www.data.act.gov.au",
            app_token=secrets.app_token,
            username=secrets.username,
            password=secrets.password)

## You can initialize the client in "severely rate-limited" form with the following
# client = Socrata("www.data.act.gov.au", None)

street_light_dataset = client.get("n9u5-bt96", limit=100000)
all_paths_dataset = client.get("ee7d-h7nm", limit=100000)

# Path Data

In [6]:
## Important quantities may be:
    ## CYCLEPATH or FOOTPATH
    ## Coordinates of each path
    ## Is path (segments of points) illuminated? boolean (may require interpolation?...)

## Boilerplate
def reverse_coordinates_list_of_lists(coordinate_list: list) -> list:
    return [[x[1], x[0]] for x in coordinate_list[0]]


all_paths_df = pd.DataFrame.from_records(all_paths_dataset)

## split out the GeoJSON dictionary into separate dataframe columns
## and concat the result back into the original dataframe
all_paths_df = pd.concat([all_paths_df.drop(['the_geom'], axis=1), all_paths_df['the_geom'].apply(pd.Series)], axis=1)

## reverse the coordinate format from [long, lat] to [lat, long]
reversed_path_coords = all_paths_df.apply(lambda x: reverse_coordinates_list_of_lists(x['coordinates']), axis=1).rename("reversed_coordinates").to_frame()

## replace old coordinate format with replaced, and clean up
all_paths_df = all_paths_df.drop(['coordinates'], axis=1)
all_paths_df = all_paths_df.merge(reversed_path_coords, how='inner', left_index=True, right_index=True)
all_paths_df = all_paths_df.rename(columns = {'reversed_coordinates':'coordinates'})

## Extract the CYCLEPATH entries (we could also do this at render-time?)
bike_paths_df = all_paths_df[all_paths_df['path_type']=='CYCLEPATH']
pedestrian_paths_df = all_paths_df[all_paths_df['path_type']=='FOOTPATH']


# Street Light Data

In [188]:
## For our first pass, we're going to make the shaky assumption that each lamp,
## irrespective of model, height, lamp_count, arm_length etc. will produce a
## uniform circle of 'acceptably-bright' light with radius X metres.

street_light_df = pd.DataFrame.from_records(street_light_dataset)
street_light_df = pd.concat([street_light_df.drop(['location'], axis=1), street_light_df['location'].apply(pd.Series)], axis=1)

street_light_location_list = []
for index, rows in street_light_df.iterrows():
    location = [float(rows.latitude), float(rows.longitude)]
    street_light_location_list.append(location)

# Mapping

In [191]:
## coordinates from Canberra wiki
initial_lat = -35.473469
initial_lng = 149.012375

# Load map centred on average coordinates
my_map = folium.Map(location=[initial_lat, initial_lng], zoom_start=10)
 
for coordinate_set in bike_paths_df['coordinates']:
    folium.PolyLine(coordinate_set, color="blue", weight=2.5, opacity=1).add_to(my_map)

#for coordinate_set in pedestrian_paths_df['coordinates']:
#    folium.PolyLine(coordinate_set, color="red", weight=2.5, opacity=1).add_to(my_map)
       
for street_light_location in street_light_location_list:
    folium.CircleMarker(street_light_location, radius=0.2,color="yellow", fill=True,fill_color="yellow",fill_opacity=0.7).add_to(my_map)

## Uncomment for pretty map
#my_map

In [ ]:
# cutoff = 5 ## arbitrary for now
# light.close_to_bike_path = False

# for path_segment in bike_path_list:
#       for light in lights_list:

#         if !light.close_to_bike_path:
            
#             P1 = path_segment[0] ## this is just a pair list of lat, long
#             P2 = path_segment[1] ## ditto

#             left_distance = math.sqrt((P1.latitude - light.latitude)**2 + (P1.longitude - light.longitude)**2)
#             if left_distance < cutoff:
#               light.close_to_bike_path = True
#               break

#             right_distance = math.sqrt((P2.latitude - light.latitude)**2 + (P2.longitude - light.longitude)**2)
#             if right_distance < cutoff:
#               light.close_to_bike_path = True
#               break

#             perp_distance = abs((P2.latitude - P1.latitude)*(P1.longitude - light.longitude) - (P1.latitude - light.latitude)*(P2.longitude - P1.longitude)) / math.sqrt((P2.latitude - P1.latitude)**2 + (P2.longitude - P1.longitude)**2)
#             if perp_distance < cutoff:
#               light.close_to_bike_path = True
#               break